In [1]:
import os
os.chdir("../")
%pwd

'/Users/akashmukherjee/Programming/LLM/Udmey/Krish Naik/textsummarizer'

In [2]:
os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"

In [3]:
from dataclasses import dataclass
from pathlib import Path
from src.constants import *
from src.utils.utlis import *
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
import torch
from datasets import load_from_disk

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

/Users/akashmukherjee/Programming/LLM/Udmey/Krish Naik/textsummarizer/my_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config=self.config.model_trainer
        params=self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config=ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.evaluation_strategy,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )
        return model_trainer_config


In [5]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        if torch.backends.mps.is_available():
            device = torch.device("mps")  # Use Metal Performance Shaders (MPS) for Apple Silicon
        else:
            device = torch.device("cpu")  # Fallback to CPU

        print(f"Using device: {device}")
        # device = "cuda" if torch.cuda.is_available() else "cpu"
        model=self.config.model_ckpt
        tokenizer=AutoTokenizer.from_pretrained(model)
        model_bart=AutoModelForSeq2SeqLM.from_pretrained(model).to(device)   
        seq_2_seq_collator=DataCollatorForSeq2Seq(tokenizer,model_bart) 
        print(f"eval_steps value: {self.config.eval_steps}")

        dataset_samsum = load_from_disk(self.config.data_path)
        training_args=TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=self.config.num_train_epochs, warmup_steps=500,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,
            evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
            gradient_accumulation_steps=4)
        
        trainer = Trainer(model=model_bart, args=training_args,
                  tokenizer=tokenizer, data_collator=seq_2_seq_collator,
                  train_dataset=dataset_samsum["test"],
                  eval_dataset=dataset_samsum["validation"])
        trainer.train()

        model_bart.save_pretrained(os.path.join(self.config.root_dir, "bart-samsum-model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))


In [6]:
# !pip install torch torchvision torchaudio --extra-index-url 
# !pip install tensorflow-macos
# !pip install tensorflow-metal


In [7]:
# !pip install --upgrade accelerate
# !pip uninstall -y transformers accelerate
# !pip install transformers accelerate

In [8]:
config = ConfigurationManager()
model_trainer_config = config.get_model_trainer_config()
model_trainer = ModelTrainer(config=model_trainer_config)
model_trainer.train()

Using device: mps
eval_steps value: steps


/Users/akashmukherjee/Programming/LLM/Udmey/Krish Naik/textsummarizer/my_env/lib/python3.11/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/var/folders/8f/zsh7smnd15z_gk8v12njmtd00000gn/T/ipykernel_97880/3270572503.py:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model_bart, args=training_args,
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss


KeyboardInterrupt: 